# Catalogação de Alfabetos
Esse notebook foi usado para a criação e envio da base de dados usada para treinar o modelo preditivo.

## Funções Auxiliares
Import de bibliotecas e definição de funções para a criação da base de dados utilizada para treinar os modelos de predição.

In [ ]:
!pip install mediapipe opencv-python

In [ ]:
import numpy as np
import mediapipe as mp
import numpy as np
import cv2
from math import floor
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

from google.colab import output

In [ ]:
def result_to_vec(results):
  """
  Transforma o objeto de análise de mãos em vetor.
  Útil para treinar modelo sktlearn.
  """
  # verifica se não há mãos no reconhecimento
  if results.multi_hand_landmarks == None:
    return None
  # se há, cria o vetor
  v = np.array([])
  for lm in results.multi_hand_landmarks[0].landmark:
    v = np.append(v, lm.x)
    v = np.append(v, lm.y)
    v = np.append(v, lm.z)
  
  return v

In [ ]:
def take_photo(letter, progress=0, total=5, filename='photo.jpg', quality=0.8):
  # Função que utiliza webcam para tirar foto para o teste
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      const progress = document.createElement('text');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Redimensiona a saída para caber no elemento de vídeo.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Aguarda o clique em Capturar.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  output.clear()
  print(f"Letra: {letter} \t\t Progresso: {progress}/{total}")
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename


In [ ]:
def detect_hands():
  cap = cv2.imread('photo.jpg', cv2.IMREAD_UNCHANGED)

  with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5,max_num_hands=1) as hands: 
  # Passar o argumento `max_num_hands` se quiser detectar mais de uma mão
          image = cv2.cvtColor(cap, cv2.COLOR_BGR2RGB)
          image.flags.writeable = False
          results = hands.process(image)

          return results

In [ ]:
def capture_alphabet(alphabet, repets):
  """
  Função que inicia o processo de captura dos alfabetos.
  Ela retorna vetores X e y para serem treinados em modelo
  de classificação.

  alphabet: list -> lista com todas as letras do alfabeto.
  repets: int -> quantas vezes cada letra será inserida.
  """
  p = 0
  total = repets * len(alphabet)

  X = np.array([])
  y = np.array([alphabet[0]])
  while p < total:
    i = 0
    while i < repets:
      # tira a foto e transforma em vetor
      idx = floor(p/repets)
      letter = alphabet[idx]
      filename = take_photo(letter=letter, progress=p, total = total)
      v = result_to_vec(detect_hands())

      # se o vetor não for nulo, salva respostas
      if v is not None:
        if len(X) == 0:
          X = v
          y = np.array([alphabet[0]])
        else:
          X = np.append(X, v)
          y = np.append(y, letter)
        p += 1 # soma 1 ao progresso
        i += 1 # soma 1 as repetições
    
  # corrige o formato dos vetores
  X = X.reshape((-1, 21*3))
  y = y

  output.clear()
  print("Catalogamento concluído! Parabéns! 🥳🥳")
  return X, y

In [ ]:
import firebase_admin
import json
from firebase_admin import credentials, firestore

cred = credentials.Certificate("/content/serviceAccountKey.json")
firebase_admin.initialize_app(cred)

db = firestore.client()

## Treinamento de Alfabetos
Seção para captura de dados para as letras dos modelos. É possível selecionar quais letras serão treinadas e quantas repetições devem ser capturadas de cada letra. 
Ao fim da seção de captura, é possível enviar os dados gerados para o banco hospedado no Firebase e/ou fazer o download para testes de eficiência dos modelos.

In [ ]:
alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'i', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v']
repets = 5

# ===== treine o alfabeto aqui =====
X, y = capture_alphabet(alphabet, repets)

Catalogamento concluído! Parabéns! 🥳🥳


In [ ]:
# ====== Envia o treinamento ======
"""
Envia o treinamento para o banco de dados.
"""
commit = input("Digite o nome para o seu commit: ") # cada envio deve ter um nome diferente
if input("Confirma o envio? [Y/n]").lower() == "y":
  db.collection('y').document(commit).set({'value':y.tolist()})
  db.collection('X').document(commit).set({'value':X.flatten().tolist()})
else:
  print("envio cancelado.")

Digite o nome para o seu commit: laguardia_1
Confirma o envio? [Y/n]y


In [ ]:
# ===== Download de alfabeto =====
"""
Usado para medir a eficácia dos diferentes modelos testados.
"""
import pickle 

with open('alphabet.data', 'wb') as f:
  pickle.dump(X, f)